# Data Analysis On Olympic Athletes (1896 - 2022)

In this notebook, I am conducting an exploratory data analysis on Olympic athletes from 1896 to 2022 using **PostgreSQL** and the datasets I created during my web scraping project (located in the `data` folder). Therefore, the obtained outputs will include data up to September 2023.
___

## 1. Import Libraries

In [1]:
import os
import pandas as pd
import psycopg2 as pg2
import warnings

warnings.filterwarnings("ignore")

## 2. Tables of the Database
### Connecting the database from Jupyter Notebook

In [2]:
mypassword = os.getenv("POSTGRESQL_PASSWORD")

try:
    conn = pg2.connect(user = "postgres", password = mypassword, database = "olympics")
    cursor = conn.cursor()
    print("Database Connection Successful")
except pg2.connector.Error as err:
    print(f"Error: '{err}'") 

Database Connection Successful


Once the database is successfully connected, let's list the name of the tables that it contains.

In [3]:
cursor.execute("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema = 'olympics'
""")

table_names = []
print('--- Tables within "olympics" database --- ')
for table in cursor:
    print(table[1])
    table_names.append(table[1])

--- Tables within "olympics" database --- 
athletes_roles
host_cities
olympic_history
noc_countries


Here are the previews of the tables under the olympics database:

In [4]:
for table in table_names:
    print("Table: ", table)
    display(pd.read_sql("SELECT * FROM olympics." + table, conn))

Table:  athletes_roles


,id,name,roles
0,131892,Meryem Erdoğan,Competed in Olympic Games
1,4300,Maurice Maina,Competed in Olympic Games
2,60239,Stanislav Tůma,Competed in Olympic Games
3,129369,Eunice Kirwa,Competed in Olympic Games
4,142670,Sinem Kurtbay,Competed in Olympic Games
...,...,...,...
155653,122196,Aleksa Šaponjić,Competed in Olympic Games
155654,52168,Zhang Yousheng,Competed in Olympic Games
155655,18974,Werner Delmes,Competed in Olympic Games • Coach
155656,126253,Tim Payne,Competed in Olympic Games


Table:  host_cities


,year,season,game,host_city
0,1896,Summer,1896 Summer Olympics,Athina
1,1900,Summer,1900 Summer Olympics,Paris
2,1904,Summer,1904 Summer Olympics,St. Louis
3,1908,Summer,1908 Summer Olympics,London
4,1912,Summer,1912 Summer Olympics,Stockholm
...,...,...,...,...
57,2010,Winter,2010 Winter Olympics,Vancouver
58,2014,Winter,2014 Winter Olympics,Sochi
59,2018,Winter,2018 Winter Olympics,PyeongChang
60,2022,Winter,2022 Winter Olympics,Beijing


Table:  olympic_history


,id,name,gender,born,died,height,weight,team,game,noc,sport,event,medal
0,131892,Meryem Erdoğan,Female,24 April 1990,None,172 cm,55 kg,Türkiye,2016 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",None
1,131892,Meryem Erdoğan,Female,24 April 1990,None,172 cm,55 kg,Türkiye,2020 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",None
2,131892,Meryem Erdoğan,Female,24 April 1990,None,172 cm,55 kg,Türkiye,2020 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",None
3,4300,Maurice Maina,Male,1 January 1963,None,158 cm,47 kg,Kenya,1988 Summer Olympics,KEN,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",None
4,4300,Maurice Maina,Male,1 January 1963,None,158 cm,47 kg,Kenya,1988 Summer Olympics,KEN,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
476343,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,None,170 cm,64 kg,United States,1988 Summer Olympics,USA,Fencing,"Fencing, Foil, Individual, Women(Olympic)",None
476344,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,None,170 cm,64 kg,United States,1988 Summer Olympics,USA,Fencing,"Fencing, Foil, Team, Women(Olympic)",None
476345,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,None,170 cm,64 kg,United States,1992 Summer Olympics,USA,Fencing,"Fencing, Foil, Individual, Women(Olympic)",None
476346,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,None,170 cm,64 kg,United States,1992 Summer Olympics,USA,Fencing,"Fencing, Foil, Team, Women(Olympic)",None


Table:  noc_countries


,noc,country
0,AFG,Afghanistan
1,ALB,Albania
2,ALG,Algeria
3,ASA,American Samoa
4,AND,Andorra
...,...,...
228,WIF,West Indies Federation
229,YEM,Yemen
230,YUG,Yugoslavia
231,ZAM,Zambia


## 3. Data Cleaning

### 3.1 Standardize inconsistent date values and Handle missing values

In both the `born` and `died` columns, the date values exhibit inconsistency. The following are the various formats found in both columns:
- `'(1880 or 1881)'` 
- `'(circa 1880)'` 
- `'(c. 1880)'`
- `'September 1880'`
- `'1933'` 
- `'NULL'`

In [5]:
# Check date values that do not start with digits
pd.read_sql("""
SELECT DISTINCT born
FROM olympics.olympic_history
WHERE (LENGTH(born) = 4 OR NOT born ~ E'^[0-9]+') AND born != 'NULL' 
""", conn)

,born
0,(1880 or 1881)
1,(1888 or 1889)
2,(1896 or 1897)
3,(1901 or 1902)
4,(1910 or 1911)
...,...
270,September 1898
271,September 1912
272,September 1929
273,September 1949


In [6]:
# Create empty table
cursor.execute("DROP TABLE IF EXISTS olympics.olympic_history_cleaned;")

cursor.execute("""
CREATE TABLE olympics.olympic_history_cleaned
(
    id INTEGER,
    name VARCHAR,
    gender VARCHAR,
    born VARCHAR,
    birth_year INTEGER,
    died VARCHAR,
    death_year INTEGER,
    height_cm INTEGER,
    weight_kg INTEGER,
    noc VARCHAR,
    game VARCHAR,
    sport VARCHAR,
    event VARCHAR,
    medal VARCHAR
);
""")


# Insert cleaned version dataset into the newly created table
cursor.execute("""
INSERT INTO olympics.olympic_history_cleaned
SELECT
    id,
    name,
    gender,
    
    -- Extract Year values from 'born' and 'died' columns 
    -- Handle missing values
    
    CASE WHEN born = 'NULL' THEN NULL ELSE born END AS born,
    CASE WHEN born = 'NULL' THEN NULL ELSE SUBSTRING(born from '\d{4}')::INTEGER END AS birth_year,
    CASE WHEN died = 'NULL' THEN NULL ELSE died END AS died,
    CASE WHEN died = 'NULL' THEN NULL ELSE SUBSTRING(died from '\d{4}')::INTEGER END AS death_year,
    
    -- Extract measurements
    
    CASE WHEN height = 'NULL' THEN NULL ELSE SUBSTRING(height from '\d{3}')::INTEGER END AS height_cm,
    CASE WHEN weight = 'NULL' THEN NULL ELSE SUBSTRING(weight from '\d{2}')::INTEGER END AS weight_kg,
    
    noc, 
    game, 
    sport, 
    event,
    CASE WHEN medal = 'NULL' THEN NULL ELSE medal END AS medal
    
FROM olympics.olympic_history 
""")


# Saving
conn.commit()

In [7]:
pd.read_sql("""
SELECT *
FROM olympics.olympic_history_cleaned
""", conn)

,id,name,gender,born,birth_year,died,death_year,height_cm,weight_kg,noc,game,sport,event,medal
0,131892,Meryem Erdoğan,Female,24 April 1990,1990.0,None,NaN,172.0,55.0,TUR,2016 Summer Olympics,Athletics,"Athletics, Marathon, Women(Olympic)",None
1,131892,Meryem Erdoğan,Female,24 April 1990,1990.0,None,NaN,172.0,55.0,TUR,2020 Summer Olympics,Athletics,"Athletics, Marathon, Women(Olympic)",None
2,131892,Meryem Erdoğan,Female,24 April 1990,1990.0,None,NaN,172.0,55.0,TUR,2020 Summer Olympics,Athletics,"Athletics, Marathon, Women(Olympic)",None
3,4300,Maurice Maina,Male,1 January 1963,1963.0,None,NaN,158.0,47.0,KEN,1988 Summer Olympics,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",None
4,4300,Maurice Maina,Male,1 January 1963,1963.0,None,NaN,158.0,47.0,KEN,1988 Summer Olympics,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476343,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,1965.0,None,NaN,170.0,64.0,USA,1988 Summer Olympics,Fencing,"Fencing, Foil, Individual, Women(Olympic)",None
476344,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,1965.0,None,NaN,170.0,64.0,USA,1988 Summer Olympics,Fencing,"Fencing, Foil, Team, Women(Olympic)",None
476345,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,1965.0,None,NaN,170.0,64.0,USA,1992 Summer Olympics,Fencing,"Fencing, Foil, Individual, Women(Olympic)",None
476346,20989,Caitlin Bilodeaux-Banos,Female,17 March 1965,1965.0,None,NaN,170.0,64.0,USA,1992 Summer Olympics,Fencing,"Fencing, Foil, Team, Women(Olympic)",None


### 3.2 Check duplicates
Using a Window function, the following code confirms that the dataset contains duplicate records.

In [8]:
pd.read_sql("""
WITH count_occurences_cte AS
(
    SELECT 
        h.*,
        ROW_NUMBER() OVER (PARTITION BY h.* ORDER BY id) AS occurence
    FROM olympics.olympic_history_cleaned h
)
SELECT *
FROM count_occurences_cte
WHERE occurence > 1
LIMIT 5
""", conn)

,id,name,gender,born,birth_year,died,death_year,height_cm,weight_kg,noc,game,sport,event,medal,occurence
0,1,Jean-François Blanchy,Male,12 December 1886,1886,2 October 1960,1960.0,NaN,NaN,FRA,1920 Summer Olympics,Tennis,"Tennis, Doubles, Men(Olympic)",None,2
1,2,Arnaud Boetsch,Male,1 April 1969,1969,None,NaN,183.0,76.0,FRA,1996 Summer Olympics,Tennis,"Tennis, Doubles, Men(Olympic)",None,2
2,3,Jean Borotra,Male,13 August 1898,1898,17 July 1994,1994.0,183.0,76.0,FRA,1924 Summer Olympics,Tennis,"Tennis, Doubles, Men(Olympic)",Bronze,2
3,4,Jacques Brugnon,Male,11 May 1895,1895,20 March 1978,1978.0,168.0,64.0,FRA,1924 Summer Olympics,Tennis,"Tennis, Doubles, Men(Olympic)",Silver,2
4,5,Albert Canet,Male,17 April 1878,1878,25 July 1930,1930.0,NaN,NaN,FRA,1912 Summer Olympics,Tennis,"Tennis, Doubles, Men(Olympic)",Bronze,2


Now, let's use `ctid` as our unique index for our dataset to remove duplicates. 

In [9]:
cursor.execute("""
WITH duplicates_cte AS 
(
    SELECT 
        ctid,
        ROW_NUMBER() OVER (PARTITION BY h.*) AS row_num
    FROM olympics.olympic_history_cleaned h
)
DELETE 
FROM olympics.olympic_history_cleaned h
WHERE h.ctid IN 
(
    SELECT ctid
    FROM duplicates_cte
    WHERE row_num > 1
);
""")

# Save
conn.commit()

## Case Study Questions

### 1. How many olympics games have been held?

In [10]:
pd.read_sql("""
SELECT COUNT(DISTINCT h.game) AS "Total Number Of Olympic Games"
FROM olympics.olympic_history_cleaned h
JOIN olympics.host_cities c ON h.game = c.game
""", conn)

,Total Number Of Olympic Games
0,53


#### 2. List down all Olympics games held so far.

In [11]:
pd.read_sql("""
SELECT DISTINCT c.year, h.game, c.host_city
FROM olympics.olympic_history_cleaned h
JOIN olympics.host_cities c ON h.game = c.game
ORDER BY c.year
""", conn)

,year,game,host_city
0,1896,1896 Summer Olympics,Athina
1,1900,1900 Summer Olympics,Paris
2,1904,1904 Summer Olympics,St. Louis
3,1908,1908 Summer Olympics,London
4,1912,1912 Summer Olympics,Stockholm
5,1920,1920 Summer Olympics,Antwerpen
6,1924,1924 Summer Olympics,Paris
7,1924,1924 Winter Olympics,Chamonix
8,1928,1928 Summer Olympics,Amsterdam
9,1928,1928 Winter Olympics,Sankt Moritz


#### 3. Mention the total number of nations who participated in each olympics game?

In [12]:
pd.read_sql("""
SELECT 
    h.game,
    c.host_city,
    COUNT(DISTINCT h.noc) AS nb_nations
FROM olympics.olympic_history_cleaned h
JOIN olympics.host_cities c ON h.game = c.game
JOIN olympics.athletes_roles r ON h.id = r.id
WHERE r.roles NOT LIKE '%(non-medal events)%'
GROUP BY h.game, c.host_city
""", conn)

,game,host_city,nb_nations
0,1896 Summer Olympics,Athina,14
1,1900 Summer Olympics,Paris,27
2,1904 Summer Olympics,St. Louis,10
3,1908 Summer Olympics,London,23
4,1912 Summer Olympics,Stockholm,29
5,1920 Summer Olympics,Antwerpen,29
6,1924 Summer Olympics,Paris,45
7,1924 Winter Olympics,Chamonix,20
8,1928 Summer Olympics,Amsterdam,46
9,1928 Winter Olympics,Sankt Moritz,25


#### 4. Which year saw the highest and lowest number of countries participating in olympics?

In [13]:
pd.read_sql("""
WITH count_nations_cte AS
( 
    SELECT h.game, COUNT(DISTINCT h.noc) AS nb_nations
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.host_cities c ON h.game = c.game
    GROUP BY h.game
)
SELECT DISTINCT
    CONCAT(FIRST_VALUE(game) OVER (ORDER BY nb_nations ASC), ' - ', 
        FIRST_VALUE(nb_nations) OVER (ORDER BY nb_nations ASC), ' nations') AS lowest_nb_nations,
    CONCAT(FIRST_VALUE(game) OVER (ORDER BY nb_nations DESC), ' - ', 
        FIRST_VALUE(nb_nations) OVER (ORDER BY nb_nations DESC), ' nations') AS highest_nb_nations
FROM count_nations_cte
""", conn)

,lowest_nb_nations,highest_nb_nations
0,1904 Summer Olympics - 10 nations,2016 Summer Olympics - 207 nations


#### 5. Which nation has participated in all of the olympic games?

In [11]:
pd.read_sql("""
SELECT COUNT(DISTINCT h.game) AS "Total Olympic Games"
FROM olympics.olympic_history_cleaned h
JOIN olympics.host_cities c ON h.game = c.game
""", conn) 

,Total Olympic Games
0,53


In [12]:
pd.read_sql("""
WITH count_games_cte AS
(
    SELECT h.country, COUNT(DISTINCT h.game) AS total_olympics_attended
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.host_cities c ON h.game = c.game
    GROUP BY h.country
)
SELECT country, total_olympics_attended
FROM count_games_cte
WHERE total_olympics_attended = 
(
    SELECT COUNT(DISTINCT h.game) AS total_games
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.host_cities c ON h.game = c.game
)
""", conn)

,country,total_olympics_attended
0,Great Britain,53
1,Italy,53


#### 6. Identify the sport which was played in all summer olympics.

In [13]:
pd.read_sql("""
SELECT COUNT(DISTINCT h.game) AS "Total Number of Summer Games"
FROM olympics.olympic_history_cleaned h
JOIN olympics.host_cities c ON h.game = c.game
WHERE c.season = 'Summer'
""", conn) 

,Total Number of Summer Games
0,29


In [14]:
pd.read_sql("""
WITH count_summer_games_cte AS
(
    SELECT h.sport, COUNT(DISTINCT h.game) AS total_summer_games_attended
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.host_cities c ON h.game = c.game
    WHERE c.season = 'Summer'
    GROUP BY h.sport
)
SELECT sport, total_summer_games_attended
FROM count_summer_games_cte
WHERE total_summer_games_attended =
(
    SELECT COUNT(DISTINCT h.game) AS "Total Number of Summer Games"
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.host_cities c ON h.game = c.game
    WHERE c.season = 'Summer'
)
""", conn) 

,sport,total_summer_games_attended
0,Artistic Gymnastics (Gymnastics),29
1,Athletics,29
2,Fencing,29
3,Swimming (Aquatics),29


#### 7. Which Sports were just played only once in the olympics?

In [27]:
pd.read_sql("""
WITH count_sport_games_cte AS
(
    SELECT 
        h.sport, 
        COUNT(DISTINCT h.game) AS "nb_games"
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.host_cities c ON h.game = c.game
    GROUP BY h.sport
    HAVING COUNT(DISTINCT h.game) = 1
)
SELECT 
    DISTINCT cte.sport, 
    h.game, 
    c.host_city, 
    cte.nb_games
FROM count_sport_games_cte cte
LEFT JOIN olympics.olympic_history_cleaned h ON cte.sport = h.sport
JOIN olympics.host_cities c ON h.game = c.game
ORDER BY h.game 
""", conn) 

,sport,game,host_city,nb_games
0,Automobile Racing,1900 Summer Olympics,Paris,1
1,Ballooning (Air Sports),1900 Summer Olympics,Paris,1
2,Boules,1900 Summer Olympics,Paris,1
3,Cricket,1900 Summer Olympics,Paris,1
4,Croquet,1900 Summer Olympics,Paris,1
5,Equestrian Driving (Equestrian),1900 Summer Olympics,Paris,1
6,Firefighting,1900 Summer Olympics,Paris,1
7,Fishing,1900 Summer Olympics,Paris,1
8,Motorcycle Sports,1900 Summer Olympics,Paris,1
9,Roque,1904 Summer Olympics,St. Louis,1


#### 8. Fetch the total number of sports played in each olympic games.

In [16]:
pd.read_sql("""
SELECT 
    h.game, 
    c.host_city, 
    COUNT(DISTINCT h.sport) AS total_number_disciplines
FROM olympics.olympic_history_cleaned h
JOIN olympics.host_cities c ON h.game = c.game
JOIN olympics.athletes_roles r ON h.id = r.id
WHERE r.roles NOT LIKE '%(non-medal events)%'
GROUP BY h.game, c.host_city
""", conn)

,game,host_city,total_number_disciplines
0,1896 Summer Olympics,Athina,10
1,1900 Summer Olympics,Paris,22
2,1904 Summer Olympics,St. Louis,18
3,1908 Summer Olympics,London,24
4,1912 Summer Olympics,Stockholm,19
5,1920 Summer Olympics,Antwerpen,29
6,1924 Summer Olympics,Paris,23
7,1924 Winter Olympics,Chamonix,10
8,1928 Summer Olympics,Amsterdam,21
9,1928 Winter Olympics,Sankt Moritz,8


#### 9. Fetch details of the oldest athletes to win a gold medal.

Notice that some of the athletes' birth and death dates are unknown. Therefore, to answer the question more accurately, I will filter out the athletes with unknown birth years from the dataset and retrieve athletes who were born from 1920.

In [69]:
pd.read_sql("""
SELECT 
    h.id, 
    h.name, 
    h.birth_year,
    h.death_year,
    CASE 
        WHEN death_year IS NOT NULL THEN death_year - birth_year
        ELSE 2023 - birth_year 
    END AS age,
    n.country,
    h.sport,
    COUNT(h.medal) AS gold_medals
    
FROM olympics.olympic_history_cleaned h
JOIN olympics.noc_countries n ON h.noc = n.noc
WHERE 
    h.medal = 'Gold' 
    AND h.birth_year > 1920
    AND h.birth_year IS NOT NULL
    AND h.event LIKE '%Olympic%'
GROUP BY h.id, h.name, h.birth_year, h.death_year, n.country, h.sport
ORDER BY age DESC
LIMIT 5
""", conn)

,id,name,birth_year,death_year,age,country,sport,gold_medals
0,28630,Ágnes Keleti,1921,NaN,102,Hungary,Artistic Gymnastics (Gymnastics),5
1,81691,Siiri Rantanen,1924,2023.0,99,Finland,Cross Country Skiing (Skiing),1
2,14004,Charles Coste,1924,NaN,99,France,Cycling Track (Cycling),1
3,19303,Amit Singh Bakshi,1925,NaN,98,India,Hockey,1
4,97517,Viktor Shuvalov,1923,2021.0,98,Soviet Union,Ice Hockey (Ice Hockey),1


#### 10. Find the Ratio of male and female athletes participated in all olympic games.

In [52]:
pd.read_sql("""
SELECT 
    TO_CHAR(SUM(CASE WHEN gender = 'Male' THEN 1 ELSE 0 END), 'FM999,999') AS nb_males,
    TO_CHAR(SUM(CASE WHEN gender = 'Female' THEN 1 ELSE 0 END), 'FM999,999') AS nb_females,
    TO_CHAR(COUNT(id), 'FM999,999') AS total_athletes,
    CONCAT(ROUND(SUM(CASE WHEN gender = 'Male' THEN 1 ELSE 0 END)/COUNT(id)::NUMERIC * 100, 1), ' %') AS males_percent,
    CONCAT(ROUND(SUM(CASE WHEN gender = 'Female' THEN 1 ELSE 0 END)/COUNT(id)::NUMERIC * 100, 1), ' %') AS females_percent,
    CONCAT(ROUND(SUM(CASE WHEN gender = 'Male' THEN 1 ELSE 0 END)/SUM(CASE WHEN gender = 'Female' THEN 1 ELSE 0 END)::NUMERIC), ' : 1') AS "Ration M:F"    
FROM 
(
    -- Retrieve unique IDs
    
    SELECT DISTINCT h.id, h.gender, h.game
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.athletes_roles r ON h.id = r.id
    WHERE r.roles NOT LIKE '%Non-starter%' AND h.event LIKE '%Olympic%'
    
) unique_id

""", conn)

,nb_males,nb_females,total_athletes,males_percent,females_percent,Ration M:F
0,"148,735","56,775","205,510",72.4 %,27.6 %,3 : 1


#### 11. In which year were female athletes permitted to participate in the Olympic Games? Provide a breakdown of the male and female participants for each Olympic Games.

In [35]:
pd.read_sql("""   
WITH unique_id_cte AS
(
    SELECT 
        DISTINCT id, 
        name, 
        gender, 
        game
    FROM 
    (
        SELECT h.id, h.name, h.gender, h.game
        FROM olympics.olympic_history_cleaned h
        JOIN olympics.athletes_roles r ON h.id = r.id
        WHERE r.roles NOT LIKE '%Non-starter%' AND h.event LIKE '%(Olympic)%'
    ) e
)
SELECT 
    game,
    SUM(CASE WHEN gender = 'Male' THEN 1 ELSE 0 END) AS nb_males,
    SUM(CASE WHEN gender = 'Female' THEN 1 ELSE 0 END) AS nb_females,
    COUNT(id) AS total_athletes
FROM unique_id_cte
GROUP BY game
ORDER BY game
""", conn)

,game,nb_males,nb_females,total_athletes
0,1896 Summer Olympics,141,0,141
1,1900 Summer Olympics,1182,22,1204
2,1904 Summer Olympics,636,6,642
3,1906 Intercalated Games,23,0,23
4,1908 Summer Olympics,1893,44,1937
5,1912 Summer Olympics,2265,54,2319
6,1920 Summer Olympics,2478,76,2554
7,1924 Summer Olympics,2999,156,3155
8,1924 Winter Olympics,282,13,295
9,1928 Summer Olympics,2888,313,3201


#### 12. In which sport female athletes first played at the Olympic Games?

In [23]:
pd.read_sql("""
SELECT 
    sport, 
    COUNT(DISTINCT h.id) AS nb_female_athletes
FROM olympics.olympic_history_cleaned h
JOIN olympics.athletes_roles r ON h.id = r.id
WHERE 
    r.roles NOT LIKE '%Non-starter%' 
    AND h.event LIKE '%(Olympic)%'
    AND h.gender = 'Female'
    AND h.game = '1900 Summer Olympics'
GROUP BY sport
ORDER BY nb_female_athletes DESC
""", conn)

,sport,nb_female_athletes
0,Golf,10
1,Tennis,5
2,Croquet,3
3,Equestrian Dressage (Equestrian),3
4,Sailing,1


#### 13. Fetch the top 5 athletes who have won the most gold medals.

In [21]:
pd.read_sql("""
SELECT 
    h.id, 
    h.name, 
    n.country, 
    COUNT(h.medal) AS gold_medals
FROM olympics.olympic_history_cleaned h
JOIN olympics.noc_countries n ON h.noc = n.noc
WHERE h.event LIKE '%Olympic%' AND h.medal = 'Gold'
GROUP BY h.id, h.name, n.country, h.noc
ORDER BY gold_medals DESC, n.country DESC
LIMIT 5
""", conn)

,id,name,country,gold_medals
0,93860,Michael Phelps,United States,23
1,51572,Mark Spitz,United States,9
2,78692,Carl Lewis,United States,9
3,29198,Larisa Latynina,Soviet Union,9
4,67728,Paavo Nurmi,Finland,9


#### 14. Fetch the top 5 athletes who have won the most medals (gold/silver/bronze).

In [19]:
pd.read_sql("""
SELECT 
    h.id, 
    h.name, 
    n.country, 
    SUM(CASE WHEN h.medal = 'Gold' THEN 1 ELSE 0 END) AS gold,
    SUM(CASE WHEN h.medal = 'Silver' THEN 1 ELSE 0 END) AS silver,
    SUM(CASE WHEN h.medal = 'Bronze' THEN 1 ELSE 0 END) AS bronze,
    COUNT(h.medal) AS total_medals
FROM olympics.olympic_history_cleaned h
JOIN olympics.noc_countries n ON h.noc = n.noc
WHERE h.event LIKE '%Olympic%'
GROUP BY h.id, h.name, n.country
ORDER BY total_medals DESC
LIMIT 5
""", conn)

,id,name,country,gold,silver,bronze,total_medals
0,93860,Michael Phelps,United States,23,3,2,28
1,29198,Larisa Latynina,Soviet Union,9,5,4,18
2,31235,Nikolay Andrianov,Soviet Union,7,5,3,15
3,101008,Marit Bjørgen,Norway,8,4,3,15
4,84154,Ole Einar Bjørndalen,Norway,8,4,1,13


#### 15. Fetch the top 5 most successful countries in olympics. Success is defined by no of medals won.

In [16]:
pd.read_sql("""
SELECT 
    n.country,
    COUNT(h.medal) AS total_medals
FROM olympics.olympic_history_cleaned h
JOIN olympics.noc_countries n ON h.noc = n.noc
WHERE h.event LIKE '%Olympic%'
GROUP BY n.country
ORDER BY total_medals DESC
LIMIT 5
""", conn)

,country,total_medals
0,United States,5952
1,Soviet Union,2480
2,Germany,2356
3,Great Britain,2179
4,France,1914


The number of Olympic medals each country has won over time seems unusually large, almost double the actual data from [Olympia.org](https://www.olympedia.org/statistics/medal/country). So, we can't just use the simple method of counting all the medal entries in the olympic_history dataset (which lists every athlete's participation, whether they won or not). If we did that, every athlete in team sports would end up with a medal, which would make the total medals count incorrect.

To solve this problem, we need to organize the data differently. We should group the data by country, specific games, events, and types of medals by using a Window function. This will help us get a more accurate count of the medals each country has actually earned.

In [14]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT 
        n.country, 
        h.game,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY n.noc, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%Olympic%' 
)
SELECT country, COUNT(medal) AS total_medals
FROM count_medals_cte
WHERE rk = 1
GROUP BY country
ORDER BY total_medals DESC
LIMIT 5
""", conn)

,country,total_medals
0,United States,2985
1,Soviet Union,1197
2,Germany,1093
3,Great Britain,975
4,France,915


#### 16. List down total gold, silver and broze medals won by each country.

In [13]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT 
        n.country,
        h.game,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY h.noc, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%Olympic%'
)
SELECT 
    country, 
    SUM(CASE WHEN medal = 'Gold' THEN 1 ELSE 0 END) AS gold,
    SUM(CASE WHEN medal = 'Silver' THEN 1 ELSE 0 END) AS silver,
    SUM(CASE WHEN medal = 'Bronze' THEN 1 ELSE 0 END) AS bronze,
    COUNT(medal) AS total_medals
FROM count_medals_cte
WHERE rk = 1
GROUP BY country
ORDER BY total_medals DESC
LIMIT 5
""", conn)

,country,gold,silver,bronze,total_medals
0,United States,1183,962,840,2985
1,Soviet Union,471,373,353,1197
2,Germany,358,374,361,1093
3,Great Britain,310,330,335,975
4,France,275,300,340,915


#### 17. List down total gold, silver and broze medals won by each country corresponding to each olympic games

In [36]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT
        h.game,
        n.country,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY h.noc, n.country, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%Olympic%'
 
)
SELECT DISTINCT 
    game,
    country,
    SUM(CASE WHEN medal = 'Gold' THEN 1 ELSE 0 END) AS gold,
    SUM(CASE WHEN medal = 'Silver' THEN 1 ELSE 0 END) AS silver,
    SUM(CASE WHEN medal = 'Bronze' THEN 1 ELSE 0 END) AS bronze,
    COUNT(medal) AS total_medals
FROM count_medals_cte 
WHERE rk = 1
GROUP BY game, country
ORDER BY game 
LIMIT 10
""", conn)

,game,country,gold,silver,bronze,total_medals
0,1896 Summer Olympics,Australia,2,0,1,3
1,1896 Summer Olympics,Austria,2,1,2,5
2,1896 Summer Olympics,Belgium,0,0,0,0
3,1896 Summer Olympics,Bulgaria,0,0,0,0
4,1896 Summer Olympics,Denmark,1,2,3,6
5,1896 Summer Olympics,France,5,4,2,11
6,1896 Summer Olympics,Germany,7,5,2,14
7,1896 Summer Olympics,Great Britain,3,3,3,9
8,1896 Summer Olympics,Greece,10,18,17,45
9,1896 Summer Olympics,Hungary,2,1,3,6


#### 18. Identify which country won the most gold, most silver, most bronze medals and the most medals in each olympic games.

In [57]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT
        h.game,
        n.country,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY h.noc, n.country, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%Olympic%' 
 
),
count_type_medals_cte AS
(
    SELECT
        game,
        country,
        SUM(CASE WHEN medal = 'Gold' THEN 1 ELSE 0 END) AS gold,
        SUM(CASE WHEN medal = 'Silver' THEN 1 ELSE 0 END) AS silver,
        SUM(CASE WHEN medal = 'Bronze' THEN 1 ELSE 0 END) AS bronze,
        COUNT(medal) AS total_medals
    FROM count_medals_cte
    WHERE rk = 1
    GROUP BY game, country

)
SELECT DISTINCT
    game,
    CONCAT(
        FIRST_VALUE(country) OVER (PARTITION BY game ORDER BY gold DESC), ' - ',
        FIRST_VALUE(gold) OVER (PARTITION BY game ORDER BY gold DESC)
    ) AS most_gold,
    
    CONCAT(
        FIRST_VALUE(country) OVER (PARTITION BY game ORDER BY silver DESC), ' - ',
        FIRST_VALUE(silver) OVER (PARTITION BY game ORDER BY silver DESC)
    ) AS most_silver,
        
    CONCAT(
        FIRST_VALUE(country) OVER (PARTITION BY game ORDER BY bronze DESC), ' - ',
        FIRST_VALUE(bronze) OVER (PARTITION BY game ORDER BY bronze DESC)
    ) AS most_bronze,
    
    CONCAT(
        FIRST_VALUE(country) OVER (PARTITION BY game ORDER BY total_medals DESC), ' - ',
        FIRST_VALUE(total_medals) OVER (PARTITION BY game ORDER BY total_medals DESC)
    ) AS most_medals
    
FROM count_type_medals_cte 
ORDER BY game
""", conn)

,game,most_gold,most_silver,most_bronze,most_medals
0,1896 Summer Olympics,United States - 11,Greece - 18,Greece - 17,Greece - 45
1,1900 Summer Olympics,France - 29,France - 39,France - 36,France - 104
2,1904 Summer Olympics,United States - 79,United States - 85,United States - 80,United States - 244
3,1906 Intercalated Games,France - 3,Germany - 2,Netherlands - 1,France - 5
4,1908 Summer Olympics,Great Britain - 56,Great Britain - 50,Great Britain - 37,Great Britain - 143
5,1912 Summer Olympics,United States - 26,Sweden - 24,United States - 19,Sweden - 64
6,1920 Summer Olympics,United States - 41,United States - 27,United States - 27,United States - 95
7,1924 Summer Olympics,United States - 45,United States - 27,United States - 27,United States - 99
8,1924 Winter Olympics,Finland - 4,Norway - 7,Norway - 6,Norway - 17
9,1928 Summer Olympics,United States - 22,United States - 18,Germany - 19,United States - 56


#### 19. Which countries have never won gold medal but have won silver/bronze medals?

In [24]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT
        n.country,
        h.game,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY h.noc, n.country, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%(Olympic)%' 
)
SELECT 
    country,
    SUM(CASE WHEN medal = 'Gold' THEN 1 ELSE 0 END) AS gold,
    SUM(CASE WHEN medal = 'Silver' THEN 1 ELSE 0 END) AS silver,
    SUM(CASE WHEN medal = 'Bronze' THEN 1 ELSE 0 END) AS bronze,
    COUNT(medal) AS total_medals
FROM count_medals_cte
WHERE rk = 1
GROUP BY country
HAVING 
    SUM(CASE WHEN medal = 'Gold' THEN 1 ELSE 0 END) = 0 AND 
    (SUM(CASE WHEN medal = 'Silver' THEN 1 ELSE 0 END) > 0 OR
    SUM(CASE WHEN medal = 'Bronze' THEN 1 ELSE 0 END) >0)
ORDER BY country 
""", conn)

,country,gold,silver,bronze,total_medals
0,Afghanistan,0,0,2,2
1,Barbados,0,0,1,1
2,Bohemia,0,1,4,5
3,Botswana,0,1,1,2
4,Burkina Faso,0,0,1,1
5,Cyprus,0,1,0,1
6,Djibouti,0,0,1,1
7,Eritrea,0,0,1,1
8,Gabon,0,1,0,1
9,Ghana,0,1,4,5


#### 20. In which sport/event Canada has won the highest number of medals.

In [20]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT
        n.country,
        h.game,
        h.sport,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY h.noc, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%(Olympic)%' 
)
SELECT 
    sport,
    COUNT(medal) AS "Total Medals Won By Canada"
FROM count_medals_cte
WHERE rk = 1 AND country = 'Canada'
GROUP BY country, sport
ORDER BY COUNT(medal) DESC 
LIMIT 5
""", conn)

,sport,Total Medals Won By Canada
0,Athletics,64
1,Swimming (Aquatics),54
2,Rowing,43
3,Speed Skating (Skating),42
4,Short Track Speed Skating (Skating),37


#### 21. Break down all olympic games where Canada won medal for Hockey and how many medals in each olympic games.

In [26]:
pd.read_sql("""
WITH count_medals_cte AS
(
    -- Partition the dataset by country, specific games, events and medals
    
    SELECT
        n.country,
        h.game,
        h.sport,
        h.event, 
        h.medal,
        ROW_NUMBER() OVER (PARTITION BY h.noc, h.game, h.event, h.medal) AS rk
    FROM olympics.olympic_history_cleaned h
    JOIN olympics.noc_countries n ON h.noc = n.noc
    WHERE h.event LIKE '%(Olympic)%' 
)
SELECT 
    game,
    SUM(CASE WHEN medal = 'Gold' THEN 1 ELSE 0 END) AS gold,
    SUM(CASE WHEN medal = 'Silver' THEN 1 ELSE 0 END) AS silver,
    SUM(CASE WHEN medal = 'Bronze' THEN 1 ELSE 0 END) AS bronze,
    COUNT(medal) AS total_medals
FROM count_medals_cte
WHERE rk = 1 AND country = 'Canada' AND sport LIKE '%Hockey%'
GROUP BY country, game
ORDER BY game
""", conn)

,game,gold,silver,bronze,total_medals
0,1920 Summer Olympics,1,0,0,1
1,1924 Winter Olympics,1,0,0,1
2,1928 Winter Olympics,1,0,0,1
3,1932 Winter Olympics,1,0,0,1
4,1936 Winter Olympics,0,1,0,1
5,1948 Winter Olympics,1,0,0,1
6,1952 Winter Olympics,1,0,0,1
7,1956 Winter Olympics,0,0,1,1
8,1960 Winter Olympics,0,1,0,1
9,1964 Summer Olympics,0,0,0,0


In [8]:
conn.close()